# Aplicación del método Bouyoucus

## Librerías de terceros

Este _Jupyter Notebook_ utiliza las librerías **pandas** y **NumPy** para ejecutar las operaciones y transformaciones que el método Bouyoucus necesita.

Por lo tanto, primero hemos de importar las librerías:

In [2]:
import numpy as np
import pandas as pd

## Constantes

Las siguientes contantes se definen para poder acceder los ficheros de entrada, sus columnas y definir los ficheros de salida:

In [3]:
# Input file
INPUT_FILE_NAME = "DATOS TFG.xlsx"
INPUT_SHEET = "Python"

# Input data columns
SAMPLE_COLUMN = "Muestra"
TIME_COLUMN = "Tiempo"
READINGS_COLUMN = "Lecturas"
CALIBRATION_READING_COLUMN = "Lo"
WEIGHT_COLUMN = "Peso"
TEMPERATURE_COLUMN = "T"
T_FACTOR_COLUMN = "factor T"
SUSPENSION_CONCENTRATION_COLUMN = "Lc"
THETA_COLUMN =  "θ"
DIAMETER_COLUMN = "Diámetro (µm)"
FIXED_DIAMETER_COLUMN = "Diámetro corregido (µm)"
WEIGHT_PERCENTAGE_COLUMN = "% Peso"

# Output
OUTPUT_FILE_NAME = "DATOS TEXTURA.xlsx"
DATA_SHEET = "Parámetros Textura"
TEXTURE_SHEET = "Clasificación Textural"

## Diagrama semilogarítmico

La funcíon `fit_by_sample` ejecuta el ajuste de los datos de una muestra al diagrama semilogaritmico. Se ejecutará sobre una serie de datos extratidos de una agrupación de `pandas`:

In [4]:
def fit_log_by_sample(data):
    """Gets the terrain type percentage."""
    x = data[FIXED_DIAMETER_COLUMN].to_numpy()
    y = data[WEIGHT_PERCENTAGE_COLUMN].to_numpy()
    try:
        coeficients = np.polyfit(np.log(x), y, 1)
        clay = coeficients[0] * np.log(2.0) + coeficients[1]
        silt = coeficients[0] * np.log(50.0) + coeficients[1] - clay
        sand = 100.0 - sum([clay, silt])
    except np.linalg.LinAlgError:
        clay = silt = sand = np.nan
    return pd.Series({"clay": clay, "silt": silt, "sand": sand})


## Triángulo textural USDA

La siguiente función implementa los discriminantes del **tríangulo textural USDA**, para obtener la textura del suelo en función del porcentaje de arcilla, limo y arena:

In [5]:

def texture_triangle(clay, silt, sand):
    """Gets the type of texture from the givin percentage of clay, silt and sand."""
    thresholds = [
        ("arcillosa", [40.0, 100.0], [0.0, 60.0], [0.0, 65.0]),
        ("franca", [7.0, 27.0], [28.0, 50.0], [23.0, 52.0]), 
        ("arcillo arenosa", [35.0, 55.0], [0.0, 20.0], [45.0, 65.0]), 
        ("franco arcillosa", [27.0, 40.0], [15.0, 53.0], [20.0, 45.0]), 
        ("franco arcillo limosa", [27.0, 40.0], [40.0, 73.0], [0.0, 20.0]), 
        ("arcillo limosa", [40.0, 60.0], [40.0, 60.0], [0.0, 20.0]), 
        ("franco arcillo arenosa", [20.0, 35.0], [0.0, 28.0], [45.0, 80.0]), 
        ("franco limosa", [0.0, 27.0], [50.0, 80.0], [0.0, 50.0]), 
        ("limosa", [0.0, 12.0], [80.0, 100.0], [0.0, 20.0]), 
        ("franco arenosa", [0.0, 20.0], [0.0, 50.0], [50.0, 70.0]), 
        ("arena franca", [0.0, 15.0], [0.0, 30.0], [70.0, 86.0]), 
        ("arenosa", [0.0, 10.0], [0.0, 14.0], [86.0, 100.0])
    ]
    for threshold in thresholds:
        if clay > threshold[1][0] and clay < threshold[1][1] and silt > threshold[2][0] and silt < threshold[2][1] and sand > threshold[3][0] and sand < threshold[3][1]:
            return threshold[0]
    return "unknown"


## Lectura de datos de entrada

Leemos los datos desde el Excel y los almacenamos en un `DataFrame` de `padas`:

In [6]:
# Read data from Excel
df = pd.read_excel(f"./input/{INPUT_FILE_NAME}", sheet_name=INPUT_SHEET)

## Datos calculados

Operamos sobre el `DataFrame` para obtener las columnas y datos derivados:

In [7]:
# Calcultes derivative data
df[TIME_COLUMN] = df[TIME_COLUMN].astype('float32')
df[SUSPENSION_CONCENTRATION_COLUMN] = df[READINGS_COLUMN] - df[CALIBRATION_READING_COLUMN]
df[THETA_COLUMN] = -0.2761 * df[READINGS_COLUMN] + 50.957
df[DIAMETER_COLUMN] = df[THETA_COLUMN] / np.sqrt(df[TIME_COLUMN])
df[FIXED_DIAMETER_COLUMN] = df[DIAMETER_COLUMN] * df[T_FACTOR_COLUMN]
df[WEIGHT_PERCENTAGE_COLUMN] = ((df[READINGS_COLUMN] - df[CALIBRATION_READING_COLUMN]) / df[WEIGHT_COLUMN]) * 100.0

## Porcentajes de arcilla, limo y arena

Agrupamos por muestra y aplicamos el ajuste del diagrama semilogarítmio para obteners los porcentajes de arcilla, limo y arena:

In [ ]:
kinds = df.groupby(SAMPLE_COLUMN).apply(fit_log_by_sample)

Una vez obtenidos los porcentajes, aplicamos el triángulo textural USDA:

In [ ]:
kinds["texture"] = kinds.apply(lambda data: texture_triangle(data["clay"], data["silt"], data["sand"]), axis=1)

## Guardar resultados

Una vez hechos los cálculos, guardamos los resultados en un fichero de Excel:

In [ ]:
with pd.ExcelWriter(f"./output/{OUTPUT_FILE_NAME}") as writer:
    df.to_excel(writer, sheet_name=DATA_SHEET)
    kinds.to_excel(writer, sheet_name=TEXTURE_SHEET)